<a href="https://colab.research.google.com/github/innovativenexusbd/AgroAI/blob/main/marketdataanalysisafterselling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Import**

In [5]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
import matplotlib.pyplot as plt

**Loading Market Data, Preparing the Data and Splitting the Data**

In [6]:
market_df = pd.read_csv('market_data.csv')
X = market_df[['Year', 'Crop']]
X = pd.get_dummies(X, columns=['Crop'], drop_first=True)  # Convert categorical variable to dummy/indicator variables
y_revenue = market_df['Revenue']
y_buyers = market_df['TotalBuyers']
y_export = market_df['TotalExport']
y_leftovers = market_df['TotalLeftovers']
X_train, X_test, y_revenue_train, y_revenue_test, y_buyers_train, y_buyers_test, y_export_train, y_export_test, y_leftovers_train, y_leftovers_test = train_test_split(X, y_revenue, y_buyers, y_export, y_leftovers, test_size=0.2, random_state=42)

**Initiatizing the Decision Tree Model and Train the Model**

In [ ]:
revenue_model = DecisionTreeRegressor(random_state=42)
buyers_model = DecisionTreeRegressor(random_state=42)
export_model = DecisionTreeRegressor(random_state=42)
leftovers_model = DecisionTreeRegressor(random_state=42)
revenue_model.fit(X_train, y_revenue_train)
buyers_model.fit(X_train, y_buyers_train)
export_model.fit(X_train, y_export_train)
leftovers_model.fit(X_train, y_leftovers_train)

**Predicting the Future Values and Categorize the Results**

In [ ]:
future_years = np.arange(2024, 2026)
future_crops = pd.DataFrame({
    'Year': np.tile(future_years, num_crops),
    'Crop': np.repeat(crops, len(future_years))
})
future_X = pd.get_dummies(future_crops, columns=['Crop'], drop_first=True)
missing_cols = set(X_train.columns) - set(future_X.columns)
for c in missing_cols:
    future_X[c] = 0
future_X = future_X[X_train.columns]
future_crops['PredictedRevenue'] = revenue_model.predict(future_X)
future_crops['PredictedBuyers'] = buyers_model.predict(future_X)
future_crops['PredictedExport'] = export_model.predict(future_X)
future_crops['PredictedLeftovers'] = leftovers_model.predict(future_X)
def categorize_result(row):
    if row['PredictedRevenue'] > 8000 and row['PredictedBuyers'] > 800 and row['PredictedExport'] > 800 and row['PredictedLeftovers'] < 20:
        return 'Very Good'
    elif row['PredictedRevenue'] > 6000 and row['PredictedBuyers'] > 600 and row['PredictedExport'] > 600 and row['PredictedLeftovers'] < 40:
        return 'Good'
    elif row['PredictedRevenue'] > 4000 and row['PredictedBuyers'] > 400 and row['PredictedExport'] > 400 and row['PredictedLeftovers'] < 60:
        return 'Average'
    elif row['PredictedRevenue'] > 2000 and row['PredictedBuyers'] > 200 and row['PredictedExport'] > 200 and row['PredictedLeftovers'] < 80:
        return 'Bad'
    else:
        return 'Very Bad'
future_crops['Result'] = future_crops.apply(categorize_result, axis=1)


**Saving the Results**

In [ ]:
future_crops.to_csv('future_market_analysis.csv', index=False)
print("Future Market Analysis:")
print(future_crops)

**Visualizing the Results**

In [ ]:
plt.figure(figsize=(12, 8))
plt.subplot(2, 2, 1)
plt.barh(future_crops['Crop'], future_crops['PredictedRevenue'], color='skyblue')
plt.xlabel('Predicted Revenue')
plt.title('Predicted Revenue by Crop')
plt.subplot(2, 2, 2)
plt.barh(future_crops['Crop'], future_crops['PredictedBuyers'], color='lightgreen')
plt.xlabel('Predicted Buyers')
plt.title('Predicted Buyers by Crop')
plt.subplot(2, 2, 3)
plt.barh(future_crops['Crop'], future_crops['PredictedExport'], color='lightcoral')
plt.xlabel('Predicted Export')
plt.title('Predicted Export by Crop')
plt.subplot(2, 2, 4)
plt.barh(future_crops['Crop'], future_crops['PredictedLeftovers'], color='gold')
plt.xlabel('Predicted Leftovers')
plt.title('Predicted Leftovers by Crop')
plt.tight_layout()
plt.show()
